In [188]:
import re
from collections import Counter

import numpy as np

In [241]:
def reduce(number, max_iter=10000):

    for i in range(max_iter):

        number, explode_flag = explode(number)

        if not explode_flag:

            number, split_flag = split(number)

        if not (explode_flag or split_flag):
            break
    
    print(i)
    return number

In [242]:
def find_regex_add(text, regex, num):
    """given a text string, a regex, and a number,
    if the regex pattern is in the string, replace the regex match with the
    match added to the number. otherwise return the string"""

    mo = regex.search(text)

    if mo:

        found_number = int(mo.group("num"))

        text = regex.sub(f"\g<left>{ found_number + num}\g<right>", text, count=1)

    return text

In [243]:
pair_regex = re.compile(r"\[(\d+),(\d+)\]")
left_regex = re.compile(r"^(?P<left>[\]\[,\d]*)(?P<num>\d+)(?P<right>[\]\[,]*)$")
right_regex = re.compile(r"^(?P<left>[\]\[,]*)(?P<num>\d+)(?P<right>[\]\[,\d]*)$")


def explode(number):

    for mo in pair_regex.finditer(number):

        char_count = Counter(number[: mo.start()])

        if char_count["["] - char_count["]"] >= 4:

            left = number[: mo.start()]
            left = find_regex_add(left, left_regex, int(mo.group(1)))

            right = number[mo.end() :]
            right = find_regex_add(right, right_regex, int(mo.group(2)))

            print("explode", left + "0" + right)

            return left + "0" + right, True

    return number, False


assert explode("[[[[[9,8],1],2],3],4]")[0] == "[[[[0,9],2],3],4]"
assert explode("[7,[6,[5,[4,[3,2]]]]]")[0] == "[7,[6,[5,[7,0]]]]"
assert explode("[[6,[5,[4,[3,2]]]],1]")[0] == "[[6,[5,[7,0]]],3]"
assert (
    explode("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]")[0]
    == "[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]"
)
assert (
    explode("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]")[0] == "[[3,[2,[8,0]]],[9,[5,[7,0]]]]"
)

explode [[[[0,9],2],3],4]
explode [7,[6,[5,[7,0]]]]
explode [[6,[5,[7,0]]],3]
explode [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
explode [[3,[2,[8,0]]],[9,[5,[7,0]]]]


In [244]:
explode("[[[[3,0],[5,3]],[4,4]],[5,5]]")

('[[[[3,0],[5,3]],[4,4]],[5,5]]', False)

In [245]:
def split_repl(matchobj):

    num = int(matchobj.group(0))

    return f"[{num//2},{num - (num//2)}]"

In [246]:
split_regex = re.compile(r"\d{2,}")


def split(number):
    """split regular number in snail_number"""

    if split_regex.search(number):

        number = split_regex.sub(split_repl, number, count=1)

        print("split", number)

        return number, True

    return number, False

In [247]:
def add(left_number, right_number):

    return reduce(f"[{left_number},{right_number}]")


assert (
    add("[[[[4,3],4],4],[7,[[8,4],9]]]", "[1,1]") == "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]"
)

explode [[[[0,7],4],[7,[[8,4],9]]],[1,1]]
explode [[[[0,7],4],[15,[0,13]]],[1,1]]
split [[[[0,7],4],[[7,8],[0,13]]],[1,1]]
split [[[[0,7],4],[[7,8],[0,[6,7]]]],[1,1]]
explode [[[[0,7],4],[[7,8],[6,0]]],[8,1]]
5


In [248]:
def part1(input_text):

    list_of_numbers = input_text.split("\n")

    total = list_of_numbers[0]

    for num in list_of_numbers[1:]:

        total = add(total, num)
        print(total)

    return total

In [251]:
test_text1 = """[1,1]
[2,2]
[3,3]
[4,4]"""

test_text2 = """[1,1]
[2,2]
[3,3]
[4,4]
[5,5]"""

test_text3 = """[1,1]
[2,2]
[3,3]
[4,4]
[5,5]
[6,6]"""

test_text4 = """[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]"""

# assert part1(test_text1) == "[[[[1,1],[2,2]],[3,3]],[4,4]]"
# assert part1(test_text2) == "[[[[3,0],[5,3]],[4,4]],[5,5]]"
# assert part1(test_text3) == "[[[[5,0],[7,4]],[5,5]],[6,6]]"
# assert part1(test_text4) == "[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]"

In [252]:
add("[[[[6,6],[6,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]]", "[2,9]")

explode [[[[0,[12,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]
explode [[[[12,0],[[12,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]
explode [[[[12,12],[0,[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]
explode [[[[12,12],[6,0]],[[[14,7],[8,9]],[8,[8,1]]]],[2,9]]
explode [[[[12,12],[6,14]],[[0,[15,9]],[8,[8,1]]]],[2,9]]
explode [[[[12,12],[6,14]],[[15,0],[17,[8,1]]]],[2,9]]
explode [[[[12,12],[6,14]],[[15,0],[115,0]]],[3,9]]
split [[[[[6,6],12],[6,14]],[[15,0],[115,0]]],[3,9]]
explode [[[[0,18],[6,14]],[[15,0],[115,0]]],[3,9]]
split [[[[0,[9,9]],[6,14]],[[15,0],[115,0]]],[3,9]]
explode [[[[9,0],[15,14]],[[15,0],[115,0]]],[3,9]]
split [[[[9,0],[[7,8],14]],[[15,0],[115,0]]],[3,9]]
explode [[[[9,7],[0,22]],[[15,0],[115,0]]],[3,9]]
split [[[[9,7],[0,[11,11]]],[[15,0],[115,0]]],[3,9]]
explode [[[[9,7],[11,0]],[[26,0],[115,0]]],[3,9]]
split [[[[9,7],[[5,6],0]],[[26,0],[115,0]]],[3,9]]
explode [[[[9,12],[0,6]],[[26,0],[115,0]]],[3,9]]
split [[[[9,[6,6]],[0,6]],[[26,0],[115,0]]],[3,9]]
explode

'[[[[6,0],[6,6]],[[6,7],[7,7]]],[[[7,7],[8,8]],[[8,8],[7,7]]]]'

In [225]:
number = "[[[[[6,6],[6,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]"

for i in range(10):
    number, explode_flag = explode(number)
    
    if explode_flag:
        print("explode", number)

explode [[[[0,[12,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]
explode [[[[12,0],[[12,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]
explode [[[[12,12],[0,[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]
explode [[[[12,12],[6,0]],[[[14,7],[8,9]],[8,[8,1]]]],[2,9]]
explode [[[[12,12],[6,14]],[[0,[15,9]],[8,[8,1]]]],[2,9]]
explode [[[[12,12],[6,14]],[[15,0],[17,[8,1]]]],[2,9]]
explode [[[[12,12],[6,14]],[[15,0],[115,0]]],[3,9]]


In [218]:
explode('[[[[0,[12,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]')

('[[[[12,0],[[12,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]', True)

In [219]:
explode('[[[[12,0],[[12,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]')

('[[[[12,12],[0,[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]', True)

In [220]:
explode('[[[[12,12],[0,[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]],[2,9]]')

('[[[[12,12],[6,0]],[[[14,7],[8,9]],[8,[8,1]]]],[2,9]]', True)

In [221]:
explode('[[[[12,12],[6,0]],[[[14,7],[8,9]],[8,[8,1]]]],[2,9]]')

('[[[[12,12],[6,14]],[[0,[15,9]],[8,[8,1]]]],[2,9]]', True)

In [222]:
explode('[[[[12,12],[6,14]],[[0,[15,9]],[8,[8,1]]]],[2,9]]')

('[[[[12,12],[6,14]],[[15,0],[17,[8,1]]]],[2,9]]', True)

In [223]:
explode('[[[[12,12],[6,14]],[[15,0],[17,[8,1]]]],[2,9]]')

('[[[[12,12],[6,14]],[[15,0],[115,0]]],[3,9]]', True)